# Week 7 -- Data Collection

This notebook contains the functions to scrape offensive player stats, week and total fantasy points for players & the defense for a team, and the defense stats. It initializes the dataframes the observations will be added to. I set up the url functions to update with the week number that will need to be manually updated (see below).

I first import all necessary libraries and set my dataframe formats.

In [1]:
import pandas as pd
import numpy as np
import requests
import urllib
from bs4 import BeautifulSoup
import time
import warnings
warnings.simplefilter('ignore')

In [2]:
%%capture

from tqdm import tqdm_notebook as tqdm
from tqdm import tnrange
tqdm().pandas()

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.1f' % x)

### Update Week Number

week_no needs to be updated for the week just ended in the football season. This will update url functions and the dataframe for the week over week fantasy points dataframe.

In [4]:
week_no = 7

### DataFrames

#### Offense

This dataframe will be used for the offensive player stats on the season.

In [5]:
df = pd.DataFrame(columns = ['Player', 'Team', 'Position', 'Age', 'Games', 'GamesStarted', 'CompletedPasses', 
                             'PassesAttempted', 'PassingYds', 'PassingTDs', 'Interceptions', 'RushingAttempts', 
                             'RushingYds', 'RushingYdspAtt', 'RushingTDs', 'Targeted', 'Receptions', 
                             'ReceivingYds', 'YdspReception', 'ReceivingTDs', 'Fumbles', 'LostFumbles', 'TtlTDs', 
                             'TwoPTConversions', 'TwoPTConversionPasses', 'FDFantasyPts', 'PositionRank', 
                             'OverallRank'])

#### Fantasy Points

These dataframes will be used when scraping the player/team fantasy points. There is one for total points and one for the week. The week column will update when the week_no variable is updated above.

In [6]:
week_fantasy = pd.DataFrame(columns = ['Player', 'Team', 'Position', 'Week_' + str(week_no)])

In [7]:
weekTTL_fantasy = pd.DataFrame(columns = ['Player', 'Team', 'Position', 'TTL'])

### Defense

This dataframe will be used to collect a team's defensive stats as a total.

In [8]:
defense_df = pd.DataFrame(columns = ['Team', 'Ttl_Pts_Allowed', 'Ttl_Offense_Plays_Allowed', 'Yds_p_Play', 'Ttl_Yds', 
                                    'Rushing_Att', 'Rushing_Yds', 'Rushing_Yds_p_Att', 'Rushing_TDs', 'Passing_Att',
                                    'Passing_Yds_p_Att', 'Completions', 'Yds_p_Completion', 'Passing_Yds', 
                                     'Passing_TDs', 'RZ_Att', 'RZ_TD', 'RZ_Percent', 'Ttl_Turnovers', 'Interceptions',
                                    'Fumbles', 'Sacks'])

In [9]:
defense_games = pd.DataFrame(columns = ['Team', 'GP'])

### Kicking

This dataframe will be used to collect kicking information.

In [10]:
kicking_df = pd.DataFrame(columns = ['PLAYER', 'POSITION', 'TEAM', 'GP', 'FGM_A', 'FG%', 'LNG', '_1_19', '_20_29', 
                                     '_30_39', '_40_49', 'OVER50', 'XPM_A', 'XP%', 'PTS'])

### Functions

In [11]:
def get_offense(url):
    """
    This function takes a url and passes it through requests.get and then uses BeautifulSoup to parse the html for 
    stats by player.
    
    Parameters:
        url: website looking to scrape for information.
        
    Returns:
        Completed dataframe with offensive player stats season to date in the NFL.
    
    """
    
    html = requests.get(url)
    soup = BeautifulSoup(html.content, 'html.parser')
    container = soup.find('tbody')
    
    for i in tqdm(range(len(container.findAll('td', {'data-stat': 'player'})))):
        name = container.findAll('td', {'data-stat': 'player'})[i].get_text().rstrip(' ')
        team = container.findAll('td', {'data-stat': 'team'})[i].get_text()
        position = container.findAll('td', {'data-stat': 'fantasy_pos'})[i].get_text()
        age = container.findAll('td', {'data-stat': 'age'})[i].get_text()
        games = container.findAll('td', {'data-stat': 'g'})[i].get_text()
        gamesstarted = container.findAll('td', {'data-stat': 'gs'})[i].get_text()
        completions = container.findAll('td', {'data-stat': 'pass_cmp'})[i].get_text()
        pass_att = container.findAll('td', {'data-stat': 'pass_att'})[i].get_text()
        pass_yds = container.findAll('td', {'data-stat': 'pass_yds'})[i].get_text()
        pass_tds = container.findAll('td', {'data-stat': 'pass_td'})[i].get_text()
        pass_int = container.findAll('td', {'data-stat': 'pass_int'})[i].get_text()
        rush_att = container.findAll('td', {'data-stat': 'rush_att'})[i].get_text()
        rush_yds = container.findAll('td', {'data-stat': 'rush_yds'})[i].get_text()
        yds_p_att = container.findAll('td', {'data-stat': 'rush_yds_per_att'})[i].get_text()
        rushing_td = container.findAll('td', {'data-stat': 'rush_td'})[i].get_text()
        targets = container.findAll('td', {'data-stat': 'targets'})[i].get_text()
        receptions = container.findAll('td', {'data-stat': 'rec'})[i].get_text()
        rec_yds = container.findAll('td', {'data-stat': 'rec_yds'})[i].get_text()
        rec_yds_rec = container.findAll('td', {'data-stat': 'rec_yds_per_rec'})[i].get_text()
        rec_td = container.findAll('td', {'data-stat': 'rec_td'})[i].get_text()
        fumbles = container.findAll('td', {'data-stat': 'fumbles'})[i].get_text()
        lost_fum = container.findAll('td', {'data-stat': 'fumbles_lost'})[i].get_text()
        ttl_td = container.findAll('td', {'data-stat': 'all_td'})[i].get_text()
        twopt_conv = container.findAll('td', {'data-stat': 'two_pt_md'})[i].get_text()
        twopt_pass = container.findAll('td', {'data-stat': 'two_pt_pass'})[i].get_text()
        fant_pts = container.findAll('td', {'data-stat': 'fanduel_points'})[i].get_text()
        pos_rank = container.findAll('td', {'data-stat': 'fantasy_rank_pos'})[i].get_text()
        overall_rank = container.findAll('td', {'data-stat': 'fantasy_rank_overall'})[i].get_text()
        
        global df
        
        df = df.append({'Player': name, 
                       'Team': team,
                       'Position': position,
                       'Age': age,
                       'Games': games,
                       'GamesStarted': gamesstarted,
                       'CompletedPasses': completions,
                       'PassesAttempted': pass_att,
                       'PassingYds': pass_yds,
                       'PassingTDs': pass_tds,
                       'Interceptions': pass_int,
                       'RushingAttempts': rush_att,
                       'RushingYds': rush_yds,
                       'RushingYdspAtt': yds_p_att,
                       'RushingTDs': rushing_td,
                       'Targeted': targets,
                       'Receptions': receptions,
                       'ReceivingYds': rec_yds,
                       'YdspReception': rec_yds_rec,
                       'ReceivingTDs': rec_td,
                       'Fumbles': fumbles,
                       'LostFumbles': lost_fum,
                       'TtlTDs': ttl_td,
                       'TwoPTConversions': twopt_conv,
                       'TwoPTConversionPasses': twopt_pass,
                       'FDFantasyPts': fant_pts,
                       'PositionRank': pos_rank,
                       'OverallRank': overall_rank}, ignore_index = True)

In [12]:
### week fantasy points

def get_lw_fantasy(url):
    """
    This function takes a url and passes it through requests.get and then uses BeautifulSoup to parse the html for 
    fantasy points by player and team when it comes to the defense.
    
    Parameters:
        url: website looking to scrape for information.
        
    Returns:
        Lists for names, teams, positions and week fantasy points that can be set equal to the columns for a 
        dataframe.
    
    """
    
    html = requests.get(url)
    soup = BeautifulSoup(html.content, 'html.parser')
    container = soup.find('tbody')
    
    names = []
    teams = []
    positions = []
    week_1 = []
    
    team_count = 1
    pos_count = 2
    fant_pts = 3
    
    for i in range(len(container.findAll('td', class_ = 'player-label'))):
        name = container.findAll('td', class_ = 'player-label')[i].get_text()
        names.append(name)
        
    for i in range(len(container.findAll('td', class_ = 'center'))):
        if team_count <= len(container.findAll('td', class_ = 'center')):
            team = container.findAll('td', class_ = 'center')[team_count].get_text()
            teams.append(team)
            team_count += 6
        
        if pos_count <= len(container.findAll('td', class_ = 'center')):
            position = container.findAll('td', class_ = 'center')[pos_count].get_text()
            positions.append(position)
            pos_count += 6
        
        if fant_pts <= len(container.findAll('td', class_ = 'center')):
            fant_pt = container.findAll('td', class_ = 'center')[fant_pts].get_text()
            week_1.append(fant_pt)
            fant_pts += 6
    
    return names, teams, positions, week_1

In [13]:
def get_defense_stats(url):
    """
    This function takes a url and passes it through requests.get and then uses BeautifulSoup to parse the html for 
    stats for a team's defense.
    
    Parameters:
        url: website looking to scrape for information.
        
    Returns:
        Completed dataframe with each team's total defensive stats season to date in the NFL.
    
    """
    html = requests.get(url)
    soup = BeautifulSoup(html.content, 'html.parser')
    container = soup.find('tbody')
    
    yds_count = 0
    att_count = 0
    yds_p_att_count = 0
    td_count = 0
    
    for i in range(len(container.findAll('span', class_ = 'hidden-xs-down'))):
                
        name = container.findAll('span', class_ = 'hidden-xs-down')[i].get_text()
        
        ttl_pts = container.findAll('td', {'data-title': 'PTS'})[i].get_text().replace('\n', '').replace(' ', '')        
        ttl_off = container.findAll('td', {'data-title': 'PLAYS'})[i].get_text().replace('\n', '').replace(' ', '')
        ttl_yds_p_play = container.findAll('td', {'data-title': 'YDS/PLAY'})[i].get_text().replace('\n', '').replace(' ', '')
        completions = container.findAll('td', {'data-title': 'COMP'})[i].get_text().replace('\n', '').replace(' ', '')
        yds_p_comp = container.findAll('td', {'data-title': 'YDS/COMP'})[i].get_text().replace('\n', '').replace(' ', '')
        rz_att = container.findAll('td', {'data-title': 'RZ ATT'})[i].get_text().replace('\n', '').replace(' ', '')
        rz_td = container.findAll('td', {'data-title': 'RZ TD'})[i].get_text().replace('\n', '').replace(' ', '')
        rz_perc = container.findAll('td', {'data-title': 'RZ %'})[i].get_text().replace('\n', '').replace(' ', '')
        turnovers = container.findAll('td', {'data-title': 'TOV'})[i].get_text().replace('\n', '').replace(' ', '')
        ints = container.findAll('td', {'data-title': 'INT'})[i].get_text().replace('\n', '').replace(' ', '')
        fumble = container.findAll('td', {'data-title': 'FUML'})[i].get_text().replace('\n', '').replace(' ', '')
        sacks = container.findAll('td', {'data-title': 'SACKS'})[i].get_text().replace('\n', '').replace(' ', '')
        
        
        if yds_count <= len(container.findAll('td', {'data-title': 'YDS'})):
            ttl_yards = container.findAll('td', {'data-title': 'YDS'})[yds_count].get_text().replace('\n', '').replace(' ', '')
            yds_count += 1
            rush_yds = container.findAll('td', {'data-title': 'YDS'})[yds_count].get_text().replace('\n', '').replace(' ', '')
            yds_count += 1
            rec_yds = container.findAll('td', {'data-title': 'YDS'})[yds_count].get_text().replace('\n', '').replace(' ', '')
            yds_count += 1
               
        if att_count <= len(container.findAll('td', {'data-title': 'ATT'})):
            rush_att = container.findAll('td', {'data-title': 'ATT'})[att_count].get_text().replace('\n', '').replace(' ', '')
            att_count += 1
            rec_att = container.findAll('td', {'data-title': 'ATT'})[att_count].get_text().replace('\n', '').replace(' ', '')
            att_count += 1

        if yds_p_att_count <= len(container.findAll('td', {'data-title': 'YDS/ATT'})):
            rush_ypa = container.findAll('td', {'data-title': 'YDS/ATT'})[yds_p_att_count].get_text().replace('\n', '').replace(' ', '')
            yds_p_att_count += 1
            rec_ypa = container.findAll('td', {'data-title': 'YDS/ATT'})[yds_p_att_count].get_text().replace('\n', '').replace(' ', '')
            yds_p_att_count += 1
            
        if td_count <= len(container.findAll('td', {'data-title': 'YDS/ATT'})):
            rush_td = container.findAll('td', {'data-title': 'TD'})[td_count].get_text().replace('\n', '').replace(' ', '')
            td_count += 1
            rec_td = container.findAll('td', {'data-title': 'TD'})[td_count].get_text().replace('\n', '').replace(' ', '')
            td_count += 1
        
        global defense_df
        defense_df = defense_df.append({'Team': name, 
                                       'Ttl_Pts_Allowed': ttl_pts,
                                       'Ttl_Offense_Plays_Allowed': ttl_off,
                                       'Yds_p_Play': ttl_yds_p_play,
                                       'Ttl_Yds': ttl_yards,
                                       'Rushing_Att': rush_att,
                                       'Rushing_Yds': rush_yds,
                                       'Rushing_Yds_p_Att': rush_ypa,
                                       'Rushing_TDs': rush_td,
                                       'Passing_Att': rec_att,
                                       'Passing_Yds_p_Att': rec_ypa,
                                       'Completions': completions,
                                       'Yds_p_Completion': yds_p_comp,
                                       'Passing_Yds': rec_yds,
                                       'Passing_TDs': rec_td,
                                       'RZ_Att': rz_att,
                                       'RZ_TD': rz_td,
                                       'RZ_Percent': rz_perc,
                                       'Ttl_Turnovers': turnovers,
                                       'Interceptions': ints,
                                       'Fumbles': fumble,
                                       'Sacks': sacks}, ignore_index = True)

In [14]:
def get_defense_g(url):
    html = requests.get(url)
    soup = BeautifulSoup(html.content, 'html.parser')
    name_container = soup.find('table', class_ = 'Table Table--align-right Table--fixed Table--fixed-left')
    games_container = soup.find('div', class_ = 'Table__ScrollerWrapper relative overflow-hidden')
    
    teams = []
    no_games = []
    
    for i in range(len(name_container.findAll('a', class_ = 'AnchorLink'))):
        if i % 2 == 0:
            team_name = name_container.findAll('a', class_ = 'AnchorLink')[i]
            team_name = str(team_name).rstrip('"/></a>').split('"')[-1]
            teams.append(team_name)
     
    count = 0
    for i in range(len(games_container.findAll('td', class_ = 'Table__TD'))):                    
        if count <= i:
            games = games_container.findAll('td', class_ = 'Table__TD')[count].get_text()
            no_games.append(games)
            count += 9
    
    
    
    global defense_games
    defense_games['Team'] = teams
    defense_games['GP'] = no_games
                   
    return defense_games

## Collecting Data

### Offense Player Stats

In [15]:
offense_url = 'https://www.pro-football-reference.com/years/2020/fantasy.htm'
get_offense(offense_url)

In [16]:
df.head()

,Player,Team,Position,Age,Games,GamesStarted,CompletedPasses,PassesAttempted,PassingYds,PassingTDs,Interceptions,RushingAttempts,RushingYds,RushingYdspAtt,RushingTDs,Targeted,Receptions,ReceivingYds,YdspReception,ReceivingTDs,Fumbles,LostFumbles,TtlTDs,TwoPTConversions,TwoPTConversionPasses,FDFantasyPts,PositionRank,OverallRank
0,Kyler Murray,ARI,QB,23,7,7,169,253,1847,13,7,65,437,6.72,7,0,0,0,,0,2,1,7,,,202.6,1,1
1,Alvin Kamara,NOR,RB,25,6,5,0,0,0,0,0,75,364,4.85,4,53,46,460,10.00,3,0,0,7,,,147.4,1,2
2,Derrick Henry,TEN,RB,26,6,6,0,0,0,0,0,143,663,4.64,7,17,10,81,8.10,0,0,0,7,,,121.4,2,3
3,Russell Wilson,SEA,QB,32,6,6,156,219,1890,22,6,29,237,8.17,0,0,0,0,,0,1,0,0,,1,183.3,2,4
4,Calvin Ridley,ATL,WR,26,7,7,0,0,0,0,0,3,14,4.67,0,64,40,615,15.38,6,0,0,6,1,,120.9,1,5


### Fantasy Pts by Week

In [17]:
fantTTL_url = 'https://www.fantasypros.com/nfl/reports/leaders/?year=2020&start=1&end=' + str(week_no)
weekTTL_fantasy['Player'], weekTTL_fantasy['Team'], weekTTL_fantasy['Position'], weekTTL_fantasy['TTL'] = get_lw_fantasy(fantTTL_url)

In [18]:
week_url = 'https://www.fantasypros.com/nfl/reports/leaders/?year=2020&start=' + str(week_no) + '&end=' + str(week_no)
week_fantasy['Player'], week_fantasy['Team'], week_fantasy['Position'], week_fantasy['Week_' + str(week_no)] = get_lw_fantasy(week_url)

#### Merging Ttl and Week Fantasy Pts DataFrames

In [19]:
fantPts = pd.merge(weekTTL_fantasy, week_fantasy, 'left', on = 'Player')
fantPts.drop(columns = ['Team_y', 'Position_y'], inplace = True)
fantPts.rename(columns = {'Team_x': 'Team', 'Position_x': 'Position'}, inplace = True)
fantPts.head()

,Player,Team,Position,TTL,Week_7
0,Kyler Murray,ARI,QB,202.5,38.1
1,Russell Wilson,SEA,QB,183.3,32.9
2,Patrick Mahomes II,KC,QB,171.4,12.0
3,Josh Allen,BUF,QB,171.1,16.4
4,Deshaun Watson,HOU,QB,159.6,24.2


### Defense Stats by Team

In [20]:
defense_url = 'https://www.lineups.com/nfl/team-stats/defense'
get_defense_stats(defense_url)

In [21]:
defense_games_url = 'https://www.espn.com/nfl/stats/team/_/view/defense'
get_defense_g(defense_games_url)

,Team,GP
0,Tampa Bay Buccaneers,6
1,Pittsburgh Steelers,5
2,Indianapolis Colts,6
3,Los Angeles Rams,6
4,San Francisco 49ers,6
5,Washington,6
6,Chicago Bears,6
7,New Orleans Saints,5
8,New England Patriots,5
9,Baltimore Ravens,6


In [22]:
defense_games['Team'][5] = 'Washington Football Team'

In [23]:
defense_url = 'https://www.lineups.com/nfl/team-stats/defense'
html = requests.get(defense_url)

In [24]:
soup = BeautifulSoup(html.content, 'html.parser')

In [25]:
container = soup.find('tbody')

container.find('span', class_ = 'hidden-xs-down')

<span _ngcontent-sc344="" class="hidden-xs-down">Pittsburgh Steelers</span>

In [26]:
defense_df.head()

,Team,Ttl_Pts_Allowed,Ttl_Offense_Plays_Allowed,Yds_p_Play,Ttl_Yds,Rushing_Att,Rushing_Yds,Rushing_Yds_p_Att,Rushing_TDs,Passing_Att,Passing_Yds_p_Att,Completions,Yds_p_Completion,Passing_Yds,Passing_TDs,RZ_Att,RZ_TD,RZ_Percent,Ttl_Turnovers,Interceptions,Fumbles,Sacks
0,Pittsburgh Steelers,94,294,4.9,"1,426",99,331,3.3,3,171,5.6,100,12.6,"1,095",9,13,8,61.5%,9,8,1,24
1,Baltimore Ravens,104,397,5.1,"2,039",143,654,4.6,4,232,5.5,149,10.5,"1,385",8,13,10,76.9%,11,3,8,22
2,Denver Broncos,110,337,5.2,"1,749",130,553,4.3,3,193,5.8,127,10.2,"1,196",7,15,7,46.7%,5,3,2,14
3,New England Patriots,110,292,5.8,"1,695",141,596,4.2,1,143,7.3,94,12.4,"1,099",9,14,7,50%,10,6,4,8
4,Arizona Cardinals,112,403,5.2,"2,077",165,718,4.4,3,221,5.7,138,10.8,"1,359",9,24,10,41.7%,7,3,4,17


In [27]:
defense_df = pd.merge(defense_df, defense_games, 'left', on = 'Team')
# defense_df.drop(columns=['Team_y'], inplace=True)
# defense_df.rename(columns = {'Team_x': 'Team'}, inplace=True)
defense_df.head()

,Team,Ttl_Pts_Allowed,Ttl_Offense_Plays_Allowed,Yds_p_Play,Ttl_Yds,Rushing_Att,Rushing_Yds,Rushing_Yds_p_Att,Rushing_TDs,Passing_Att,Passing_Yds_p_Att,Completions,Yds_p_Completion,Passing_Yds,Passing_TDs,RZ_Att,RZ_TD,RZ_Percent,Ttl_Turnovers,Interceptions,Fumbles,Sacks,GP
0,Pittsburgh Steelers,94,294,4.9,"1,426",99,331,3.3,3,171,5.6,100,12.6,"1,095",9,13,8,61.5%,9,8,1,24,5
1,Baltimore Ravens,104,397,5.1,"2,039",143,654,4.6,4,232,5.5,149,10.5,"1,385",8,13,10,76.9%,11,3,8,22,6
2,Denver Broncos,110,337,5.2,"1,749",130,553,4.3,3,193,5.8,127,10.2,"1,196",7,15,7,46.7%,5,3,2,14,5
3,New England Patriots,110,292,5.8,"1,695",141,596,4.2,1,143,7.3,94,12.4,"1,099",9,14,7,50%,10,6,4,8,5
4,Arizona Cardinals,112,403,5.2,"2,077",165,718,4.4,3,221,5.7,138,10.8,"1,359",9,24,10,41.7%,7,3,4,17,6


In [28]:
defense_df.columns

Index(['Team', 'Ttl_Pts_Allowed', 'Ttl_Offense_Plays_Allowed', 'Yds_p_Play',
       'Ttl_Yds', 'Rushing_Att', 'Rushing_Yds', 'Rushing_Yds_p_Att',
       'Rushing_TDs', 'Passing_Att', 'Passing_Yds_p_Att', 'Completions',
       'Yds_p_Completion', 'Passing_Yds', 'Passing_TDs', 'RZ_Att', 'RZ_TD',
       'RZ_Percent', 'Ttl_Turnovers', 'Interceptions', 'Fumbles', 'Sacks',
       'GP'],
      dtype='object')

In [29]:
defense_df = defense_df[['Team', 'GP', 'Ttl_Pts_Allowed', 'Ttl_Offense_Plays_Allowed', 'Yds_p_Play', 'Ttl_Yds', 
                         'Rushing_Att', 'Rushing_Yds', 'Rushing_Yds_p_Att', 'Rushing_TDs', 'Passing_Att', 
                         'Passing_Yds_p_Att', 'Completions', 'Yds_p_Completion', 'Passing_Yds', 'Passing_TDs', 
                         'RZ_Att', 'RZ_TD', 'RZ_Percent', 'Ttl_Turnovers', 'Interceptions', 'Fumbles', 'Sacks']]
defense_df.head()

,Team,GP,Ttl_Pts_Allowed,Ttl_Offense_Plays_Allowed,Yds_p_Play,Ttl_Yds,Rushing_Att,Rushing_Yds,Rushing_Yds_p_Att,Rushing_TDs,Passing_Att,Passing_Yds_p_Att,Completions,Yds_p_Completion,Passing_Yds,Passing_TDs,RZ_Att,RZ_TD,RZ_Percent,Ttl_Turnovers,Interceptions,Fumbles,Sacks
0,Pittsburgh Steelers,5,94,294,4.9,"1,426",99,331,3.3,3,171,5.6,100,12.6,"1,095",9,13,8,61.5%,9,8,1,24
1,Baltimore Ravens,6,104,397,5.1,"2,039",143,654,4.6,4,232,5.5,149,10.5,"1,385",8,13,10,76.9%,11,3,8,22
2,Denver Broncos,5,110,337,5.2,"1,749",130,553,4.3,3,193,5.8,127,10.2,"1,196",7,15,7,46.7%,5,3,2,14
3,New England Patriots,5,110,292,5.8,"1,695",141,596,4.2,1,143,7.3,94,12.4,"1,099",9,14,7,50%,10,6,4,8
4,Arizona Cardinals,6,112,403,5.2,"2,077",165,718,4.4,3,221,5.7,138,10.8,"1,359",9,24,10,41.7%,7,3,4,17


### Pickle DataFrames

In [30]:
df.to_pickle('player_stats')
fantPts.to_pickle('fantasy_weeks')
defense_df.to_pickle('defense_data')